In [1]:
import logging

from neo4j import GraphDatabase

import gevopy as ea
from examples.algorimths import BasicPonderated as MyAlgorithm
from examples.evaluation import MostOnes as MyFitness
from examples.genotypes import Bacteria as MyGenotype
from gevopy.database import Neo4jInterface


In [2]:
LOG_FORMAT = "%(asctime)s - %(name)s - %(levelname)s\n  %(message)s"
logging.basicConfig(level="INFO", format=LOG_FORMAT)
logger = logging.getLogger("gevopy.Experiment")
logger.setLevel(logging.INFO)  # Only Experiment INFO


#### Connect to the database
Connect to a database by creating a Driver object and providing a URL and an authentication token. Once you have a Driver instance, use the .verify_connectivity() method to ensure that a working connection can be established.


In [3]:
# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "bolt:"  # "<URI for Neo4j database>"
AUTH = ""  # ("<Username>", "<Password>")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()


### Create and run the experiment
Execute an experiment by creating an instance and assigning the argument database with the driver. The experiment will automaticaly manage sessions and use the methods execute_read() and execute_write().

In [4]:
experiment = ea.Experiment(
    fitness=MyFitness(cache=True, scheduler="synchronous"),
    algorithm=MyAlgorithm(survival_rate=0.2),
    database=Neo4jInterface(URI, auth=AUTH),
)

with experiment.session() as session:
    session.add_phenotypes([MyGenotype() for _ in range(12)])
    statistics = session.run(max_score=10)


2022-12-15 08:12:53,980 - gevopy.Experiment - INFO
  [12bb1dc6-4cd7-4f5c-a0a5-cdc5310f9f1c]: [gen:0]: Start of evolutionary experiment execution
2022-12-15 08:12:54,025 - gevopy.Experiment - INFO
  [12bb1dc6-4cd7-4f5c-a0a5-cdc5310f9f1c]: [gen:1]: Completed cycle; 9
2022-12-15 08:12:54,055 - gevopy.Experiment - INFO
  [12bb1dc6-4cd7-4f5c-a0a5-cdc5310f9f1c]: [gen:2]: Completed cycle; 9
2022-12-15 08:12:54,091 - gevopy.Experiment - INFO
  [12bb1dc6-4cd7-4f5c-a0a5-cdc5310f9f1c]: [gen:3]: Completed cycle; 9
2022-12-15 08:12:54,121 - gevopy.Experiment - INFO
  [12bb1dc6-4cd7-4f5c-a0a5-cdc5310f9f1c]: [gen:4]: Completed cycle; 9
2022-12-15 08:12:54,153 - gevopy.Experiment - INFO
  [12bb1dc6-4cd7-4f5c-a0a5-cdc5310f9f1c]: [gen:5]: Completed cycle; 10
2022-12-15 08:12:54,154 - gevopy.Experiment - INFO
  [12bb1dc6-4cd7-4f5c-a0a5-cdc5310f9f1c]: [gen:5]: Experiment execution completed successfully


In [5]:
statistics


Evolutionary algorithm execution report:
  Executed generations: 5
  Best phenotype: 9d028152-4339-4a11-8b1b-918b3671afb3
  Best score: 10

### Close connections
Unless you created them using the with statement, call the .close() method on all Driver and Session instances out of the experiment to release any resources still held by them.

In [6]:
experiment.close()
